# Step 0.0. Install LightAutoML

# Step 0.1. Import necessary libraries 

In [3]:
# Standard python libraries
import logging
import os
import time
import requests
logging.basicConfig(format='[%(asctime)s] (%(levelname)s): %(message)s', level=logging.INFO)

# Installed libraries
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
import torch

# Imports from our package
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.tasks import Task

import multiprocessing as mp

# set option of max 100 displayed rows
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

[2021-09-26 19:13:15,254] (WARNING): C:\Users\User\AppData\Local\Programs\Python\Python37\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)



# Step 0.2. Parameters 

In [4]:
N_THREADS = mp.cpu_count() # threads cnt for lgbm and linear models
N_FOLDS = 10 # folds cnt for AutoML - 10 is better
RANDOM_STATE = 42 # fixed random state for various reasons
TIMEOUT = 3*3600 # Time in seconds for automl run
TARGET_NAME = 'final_price' # Target column name

# Step 0.3. Data load 

In [5]:
%%time

train_data = pd.read_csv('train_data.csv')
train_data.head()

Wall time: 69.3 ms


,row_ID,vehicle_manufacturer,vehicle_model,vehicle_category,current_mileage,vehicle_year,vehicle_gearbox_type,doors_cnt,wheels,vehicle_color,vehicle_interior_color,car_vin,car_leather_interior,deal_type,final_price
0,0,TOYOTA,Aqua s,Sedan,133000,2014,Automatic,4/5,Right-hand drive,Silver,Black,NaN,0,For Sale,3650.0
1,1,MERCEDES-BENZ,C 220,Sedan,24500,2010,Manual,4/5,Left wheel,Silver,Black,NaN,0,For Sale,6800.0
2,2,HYUNDAI,Veloster,Hatchback,31000,2016,Tiptronic,2/3,Left wheel,Silver,Black,KMHTC6AE3GU293912,1,For Sale,6300.0
3,3,HYUNDAI,Santa FE,Jeep,115459,2015,Automatic,4/5,Left wheel,Blue,Black,NaN,1,For Sale,14488.0
4,4,TOYOTA,CHR,Jeep,18950,2019,Automatic,4/5,Left wheel,Black,NaN,JTNKHMBX7K1030253,1,For Sale,5000.0


In [6]:
test_data = pd.read_csv('test_data.csv')
test_data.head()

,row_ID,vehicle_manufacturer,vehicle_model,vehicle_category,current_mileage,vehicle_year,vehicle_gearbox_type,doors_cnt,wheels,vehicle_color,vehicle_interior_color,car_vin,car_leather_interior,deal_type
0,35000,TOYOTA,Prius,Hatchback,323733,2012,Automatic,4/5,Left wheel,Grey,Black,JTDKN3DU6C5439638,1,For Sale
1,35001,HYUNDAI,Elantra,Sedan,112000,2013,Tiptronic,4/5,Left wheel,Grey,Black,SURATSHIA,1,For Sale
2,35002,LEXUS,NX 300,Jeep,16920,2018,Automatic,NaN,Left wheel,Brown,NaN,JTJYARBZ5J2104521,1,For Sale
3,35003,LEXUS,CT 200h,Hatchback,302742,2012,Automatic,4/5,Left wheel,White,NaN,JTHKD5BH4C2070945,1,For Sale
4,35004,TOYOTA,RAV 4,Jeep,1800,2002,Manual,4/5,Left wheel,Silver,Black,NaN,0,For Sale


In [8]:
submission = pd.read_csv('sample_submission.csv')
submission.head()

,row_ID,final_price
0,35000,0
1,35001,0
2,35002,0
3,35003,0
4,35004,0


# Step 0.4. Some user feature preparation 

Be creative :)

At first let's check correctness of data in train and test tables. Let's start from *current_milage*

train_data

In [9]:
train_data['current_mileage'].sort_values()

32771             0
3136              0
11941             0
1193              0
15936             0
            ...    
10885    2147483647
28857    2147483647
316      2147483647
26303    2147483647
13926    2147483647
Name: current_mileage, Length: 35000, dtype: int64

test_data

In [10]:
test_data['current_mileage'].sort_values()

5195             0
3936             0
2552             0
2550             0
3944             0
           ...    
1052    2147483647
9805    2147483647
7876    2147483647
1072    2147483647
1619    2147483647
Name: current_mileage, Length: 10697, dtype: int64

As we can see, some milage values are extremely high. World record of car mileage is around 4890000 kilometers. So let's consider milage more than 4890000 kilometers as wrong data and add this feature to our train and test datasets.

*final_price*

In [11]:
train_data.head()

,row_ID,vehicle_manufacturer,vehicle_model,vehicle_category,current_mileage,vehicle_year,vehicle_gearbox_type,doors_cnt,wheels,vehicle_color,vehicle_interior_color,car_vin,car_leather_interior,deal_type,final_price
0,0,TOYOTA,Aqua s,Sedan,133000,2014,Automatic,4/5,Right-hand drive,Silver,Black,NaN,0,For Sale,3650.0
1,1,MERCEDES-BENZ,C 220,Sedan,24500,2010,Manual,4/5,Left wheel,Silver,Black,NaN,0,For Sale,6800.0
2,2,HYUNDAI,Veloster,Hatchback,31000,2016,Tiptronic,2/3,Left wheel,Silver,Black,KMHTC6AE3GU293912,1,For Sale,6300.0
3,3,HYUNDAI,Santa FE,Jeep,115459,2015,Automatic,4/5,Left wheel,Blue,Black,NaN,1,For Sale,14488.0
4,4,TOYOTA,CHR,Jeep,18950,2019,Automatic,4/5,Left wheel,Black,NaN,JTNKHMBX7K1030253,1,For Sale,5000.0


In [12]:
train_data[train_data['final_price'] > 100000].sort_values('final_price')

,row_ID,vehicle_manufacturer,vehicle_model,vehicle_category,current_mileage,vehicle_year,vehicle_gearbox_type,doors_cnt,wheels,vehicle_color,vehicle_interior_color,car_vin,car_leather_interior,deal_type,final_price
20577,20577,BMW,520,Sedan,536688566,1992,Manual,4/5,Left wheel,Grey,Black,NaN,1,For Sale,111111.0
19820,19820,PORSCHE,Panamera Turbo S E-Hyb,Sedan,16000,2017,Tiptronic,4/5,Left wheel,Carnelian red,White,NaN,1,For Sale,135000.0
32937,32937,LAMBORGHINI,Gallardo,Coupe,26000,2013,Tiptronic,2/3,Left wheel,Yellow,Black,NaN,1,For Sale,135000.0
15291,15291,MERCEDES-BENZ,G 65 AMG 63AMG,Jeep,0,2020,Tiptronic,4/5,Left wheel,Black,Brown,NaN,1,For Sale,200000.0
12136,12136,BMW,328,Sedan,246000,2008,Automatic,4/5,Left wheel,Sky blue,Yellow,NaN,1,For Sale,1111111.0
24980,24980,SUBARU,Legacy,Sedan,97000,2003,Manual,4/5,Right-hand drive,Silver,Black,NaN,1,For Sale,1111111.0
34052,34052,MERCEDES-BENZ,ML 320,Jeep,111111111,2002,Tiptronic,4/5,Left wheel,White,Black,NaN,1,For Sale,6666666.0
3877,3877,VOLKSWAGEN,Golf,Sedan,7676,2007,Tiptronic,4/5,Left wheel,Red,Golden,NaN,0,For Sale,8388607.0
32442,32442,OPEL,Combo,Goods wagon,99999,1999,Manual,2/3,Left wheel,Blue,Black,NaN,0,For Sale,8388607.0


As we can see, prices of some cars are too high. For example Volkswagen Golf can't have the price over 8 million dollars. So let's drop all cars with price more than $200000.

In [13]:
drop_index = train_data[train_data['final_price'] > 200000].index
train_data.drop(index=drop_index, inplace=True)

Add new features

In [14]:
%%time

# define if mileage is wrong
def wrong_milage(col):
    if col > 4890000:
        return 1
    return 0

# define mileage of car per year
def milage_per_year(row):
    if row['vehicle_year'] == 0:
        return 0
    return row['current_mileage']/(2021-row['vehicle_year'])

# add manufacturer country for each car
usa     = ['FORD', 'CHEVROLET', 'JEEP', 'DODGE', 'MERCURY', 'CADILLAC', 'CHRYSLER', 
           'TESLA', 'SCION', 'GMC', 'HUMMER', 'LINCOLN', 'BUICK', 'SATURN', 'PONTIAC', 'FOTON']
japan   = ['TOYOTA', 'MITSUBISHI', 'NISSAN', 'LEXUS', 'HONDA', 'SUBARU', 'MAZDA', 
           'SUZUKI', 'ACURA', 'INFINITI', 'ISUZU', 'DAIHATSU']
england = ['MINI', 'JAGUAR', 'LAND ROVER', 'ASTON MARTIN', 'ROVER', 'MG', 'BENTLEY', 
           'ROLLS-ROYCE']
italy   = ['FIAT', 'ALFA ROMEO', 'FERRARI', 'MASERATI', 'LANCIA', 'LAMBORGHINI']
germany = ['MERCEDES-BENZ', 'BMW', 'AUDI', 'VOLKSWAGEN', 'OPEL', 'PORSCHE']
korea   = ['HYUNDAI', 'DAEWOO', 'SSANGYONG', 'KIA']
russia  = ['VAZ', 'ZAZ', 'UAZ', 'GAZ', 'MOSKVICH']
france  = ['RENAULT', 'PEUGEOT', 'CITROEN']
china   = ['GREATWALL', 'JAC']
sweden  = ['VOLVO', 'SAAB']
spain   = ['SEAT']
india   = ['TATA']
others  = ['სხვა']

def manufacturer_country(col):
    if col in usa:
        return 'USA'
    if col in japan:
        return 'Japan'
    if col in england:
        return 'England'
    if col in italy:
        return 'Italy'
    if col in germany:
        return 'Germany'
    if col in korea:
        return 'Korea'
    if col in russia:
        return 'Russia'
    if col in france:
        return 'France'
    if col in china:
        return 'China'
    if col in sweden:
        return 'Sweden'
    if col in spain:
        return 'Spain'
    if col in india:
        return 'India'
    if col in others:
        return 'Others'
    return np.nan

def create_extra_features(data):
    # add feature which shows cars that have too big mileage
    data['wrong_mileage'] = data['current_mileage'].apply(wrong_milage)
    # add feature with manufacturer country
    data['manufacturer_country'] = data['vehicle_manufacturer'].apply(manufacturer_country)
    # add squared mileage
    data['current_mileage_sq'] = data['current_mileage']**2
    # add number of NaN data in each row
    data['NANs_cnt'] = data.isnull().sum(axis = 1) 
    # add mileage per year feature   
    data['mileage_per_year'] = data.apply(milage_per_year, axis=1)
    # add squared mileage per year
    data['mileage_per_year_sq'] = data['mileage_per_year']**2

create_extra_features(train_data)
create_extra_features(test_data)

Wall time: 484 ms


Features from strong baseline

In [15]:
def create_col_with_min_freq(data, col, min_freq = 10):
    # replace rare values (less than min_freq rows) in feature by RARE_VALUE
    data[col + '_fixed'] = data[col].astype(str)
    data.loc[data[col + '_fixed'].value_counts()[data[col + '_fixed']].values < min_freq, col + '_fixed'] = "RARE_VALUE"
    data.replace({'nan': np.nan}, inplace = True)
    
def create_gr_feats(data):
    # create aggregation feats for numeric features based on categorical ones
    for cat_col in ['vehicle_manufacturer', 'vehicle_model', 'vehicle_category',
                   'vehicle_gearbox_type', 'doors_cnt', 'wheels', 'vehicle_color', 
                   'vehicle_interior_color', 'deal_type']:
        create_col_with_min_freq(data, cat_col, 15)
        for num_col in ['current_mileage', 'vehicle_year', 'car_leather_interior']:
            for n, f in [('mean', np.mean), ('min', np.nanmin), ('max', np.nanmax)]:
                data['FIXED_' + n + '_' + num_col + '_by_' + cat_col] = data.groupby(cat_col + '_fixed')[num_col].transform(f)
                
    # create features with counts
    for col in ['vehicle_manufacturer', 'vehicle_model', 'vehicle_category', 'mileage_per_year',
               'current_mileage', 'vehicle_year', 'vehicle_gearbox_type', 'doors_cnt',
               'wheels', 'vehicle_color', 'vehicle_interior_color', 'car_vin', 'deal_type']:
        data[col + '_cnt'] = data[col].map(data[col].value_counts(dropna = False))
        

all_df = pd.concat([train_data, test_data]).reset_index(drop = True)
create_gr_feats(all_df)
train_data, test_data = all_df[:len(train_data)], all_df[len(train_data):]

In [16]:
train_data.head(10)

,row_ID,vehicle_manufacturer,vehicle_model,vehicle_category,current_mileage,vehicle_year,vehicle_gearbox_type,doors_cnt,wheels,vehicle_color,vehicle_interior_color,car_vin,car_leather_interior,deal_type,final_price,wrong_mileage,manufacturer_country,current_mileage_sq,NANs_cnt,mileage_per_year,mileage_per_year_sq,vehicle_manufacturer_fixed,FIXED_mean_current_mileage_by_vehicle_manufacturer,FIXED_min_current_mileage_by_vehicle_manufacturer,FIXED_max_current_mileage_by_vehicle_manufacturer,FIXED_mean_vehicle_year_by_vehicle_manufacturer,FIXED_min_vehicle_year_by_vehicle_manufacturer,FIXED_max_vehicle_year_by_vehicle_manufacturer,FIXED_mean_car_leather_interior_by_vehicle_manufacturer,FIXED_min_car_leather_interior_by_vehicle_manufacturer,FIXED_max_car_leather_interior_by_vehicle_manufacturer,vehicle_model_fixed,FIXED_mean_current_mileage_by_vehicle_model,FIXED_min_current_mileage_by_vehicle_model,FIXED_max_current_mileage_by_vehicle_model,FIXED_mean_vehicle_year_by_vehicle_model,FIXED_min_vehicle_year_by_vehicle_model,FIXED_max_vehicle_year_by_vehicle_model,FIXED_mean_car_leather_interior_by_vehicle_model,FIXED_min_car_leather_interior_by_vehicle_model,FIXED_max_car_leather_interior_by_vehicle_model,vehicle_category_fixed,FIXED_mean_current_mileage_by_vehicle_category,FIXED_min_current_mileage_by_vehicle_category,FIXED_max_current_mileage_by_vehicle_category,FIXED_mean_vehicle_year_by_vehicle_category,FIXED_min_vehicle_year_by_vehicle_category,FIXED_max_vehicle_year_by_vehicle_category,FIXED_mean_car_leather_interior_by_vehicle_category,FIXED_min_car_leather_interior_by_vehicle_category,FIXED_max_car_leather_interior_by_vehicle_category,vehicle_gearbox_type_fixed,FIXED_mean_current_mileage_by_vehicle_gearbox_type,FIXED_min_current_mileage_by_vehicle_gearbox_type,FIXED_max_current_mileage_by_vehicle_gearbox_type,FIXED_mean_vehicle_year_by_vehicle_gearbox_type,FIXED_min_vehicle_year_by_vehicle_gearbox_type,FIXED_max_vehicle_year_by_vehicle_gearbox_type,FIXED_mean_car_leather_interior_by_vehicle_gearbox_type,FIXED_min_car_leather_interior_by_vehicle_gearbox_type,FIXED_max_car_leather_interior_by_vehicle_gearbox_type,doors_cnt_fixed,FIXED_mean_current_mileage_by_doors_cnt,FIXED_min_current_mileage_by_doors_cnt,FIXED_max_current_mileage_by_doors_cnt,FIXED_mean_vehicle_year_by_doors_cnt,FIXED_min_vehicle_year_by_doors_cnt,FIXED_max_vehicle_year_by_doors_cnt,FIXED_mean_car_leather_interior_by_doors_cnt,FIXED_min_car_leather_interior_by_doors_cnt,FIXED_max_car_leather_interior_by_doors_cnt,wheels_fixed,FIXED_mean_current_mileage_by_wheels,FIXED_min_current_mileage_by_wheels,FIXED_max_current_mileage_by_wheels,FIXED_mean_vehicle_year_by_wheels,FIXED_min_vehicle_year_by_wheels,FIXED_max_vehicle_year_by_wheels,FIXED_mean_car_leather_interior_by_wheels,FIXED_min_car_leather_interior_by_wheels,FIXED_max_car_leather_interior_by_wheels,vehicle_color_fixed,FIXED_mean_current_mileage_by_vehicle_color,FIXED_min_current_mileage_by_vehicle_color,FIXED_max_current_mileage_by_vehicle_color,FIXED_mean_vehicle_year_by_vehicle_color,FIXED_min_vehicle_year_by_vehicle_color,FIXED_max_vehicle_year_by_vehicle_color,FIXED_mean_car_leather_interior_by_vehicle_color,FIXED_min_car_leather_interior_by_vehicle_color,FIXED_max_car_leather_interior_by_vehicle_color,vehicle_interior_color_fixed,FIXED_mean_current_mileage_by_vehicle_interior_color,FIXED_min_current_mileage_by_vehicle_interior_color,FIXED_max_current_mileage_by_vehicle_interior_color,FIXED_mean_vehicle_year_by_vehicle_interior_color,FIXED_min_vehicle_year_by_vehicle_interior_color,FIXED_max_vehicle_year_by_vehicle_interior_color,FIXED_mean_car_leather_interior_by_vehicle_interior_color,FIXED_min_car_leather_interior_by_vehicle_interior_color,FIXED_max_car_leather_interior_by_vehicle_interior_color,deal_type_fixed,FIXED_mean_current_mileage_by_deal_type,FIXED_min_current_mileage_by_deal_type,FIXED_max_current_mileage_by_deal_type,FIXED_mean_vehicle_year_by_deal_type,FIXED_min_vehicle_year_by_deal_type,

# ========= AutoML preset usage =========


## Step 1. Create Task

In [17]:
%%time

task = Task('reg', loss='mae', metric='mae')

sklearn doesn't support in general case mae and will not be used.


Wall time: 2.52 ms


## Step 2. Setup columns roles

Roles setup here set target column and base date, which is used to calculate date differences:

In [18]:
%%time

roles = {'target': TARGET_NAME,
         'drop': ['row_ID']
         }

Wall time: 0 ns


## Step 3. Create AutoML from preset

To create AutoML model here we use `TabularAutoML` preset, which looks like:

![TabularAutoML preset pipeline](https://github.com/sberbank-ai-lab/LightAutoML/raw/master/imgs/tutorial_2_pipeline.png)

All params we set above can be send inside preset to change its configuration:

In [19]:
#### %%time 
automl = TabularUtilizedAutoML(task = task, 
                       timeout = TIMEOUT,
                       cpu_limit = N_THREADS,
                       general_params = {'use_algos': [['linear_l2', 'lgb', 'lgb_tuned']]},
                       reader_params = {'n_jobs': N_THREADS, 'cv': N_FOLDS, 'random_state': RANDOM_STATE},
                      )
oof_pred = automl.fit_predict(train_data, roles = roles)
logging.info('oof_pred:\n{}\nShape = {}'.format(oof_pred, oof_pred.shape))

Current random state: {'reader_params': {'random_state': 42}, 'general_params': {'return_all_predictions': False}}
Found reader_params in kwargs, need to combine
Merged variant for reader_params = {'n_jobs': 8, 'cv': 10, 'random_state': 42}
Found general_params in kwargs, need to combine
Merged variant for general_params = {'use_algos': [['linear_l2', 'lgb', 'lgb_tuned']], 'return_all_predictions': False}
Start automl preset with listed constraints:
- time: 10799.997000217438 seconds
- cpus: 8 cores
- memory: 16 gb

Train data shape: (34995, 124)
Feats was rejected during automatic roles guess: []


Layer 1 ...
Train process start. Time left 10789.108889579773 secs


[2021-09-26 19:14:42,682] (WARNING): C:\Users\User\AppData\Local\Programs\Python\Python37\lib\site-packages\numpy\core\fromnumeric.py:87: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)



Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...

===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LinearL2 =====

Linear model: C = 1e-05 score = -3611.4078763950893
Linear model: C = 5e-05 score = -3602.1437322823663
Linear model: C = 0.0001 score = -3590.31772593471
Linear model: C = 0.0005 score = -3501.52542905971
Linear model: C = 0.001 score = -3400.464418736049
Linear model: C = 0.005 score = -2868.888834524972
Linear model: C = 0.01 score = -2604.6120405829292
Linear model: C = 0.05 score = -2311.8759727232796
Linear model: C = 0.1 score = -2247.043958149501
Linear model: C = 0.5 score = -2160.126519141061
Linear model: C = 1 score = -2160.1264875368392
Linear model: C = 5 score = -2115.0022644391743
Linear model: C = 10 score = -2115.0023634272984
Linear model: C = 50 score = -2115.00235206604

===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_LinearL2 =====

Linear model: C = 1e-05 score = -3615.91643031529
Linear model: C = 5e-05 score = -3606.7017613699777
Linear m

Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[2021-09-26 19:17:51,162] (INFO): A new study created in memory with name: no-name-6da3b376-6822-41e1-962d-48da63e7626a


Start fitting Lvl_0_Pipe_1_Mod_1_LightGBM ...

===== Start working with fold 0 for Lvl_0_Pipe_1_Mod_1_LightGBM =====

Training until validation scores don't improve for 200 rounds
[100]	valid's l1: 1518.88
[200]	valid's l1: 1382.67
[300]	valid's l1: 1356.21
[400]	valid's l1: 1345.81
[500]	valid's l1: 1340.3
[600]	valid's l1: 1333.27
[700]	valid's l1: 1328.47
[800]	valid's l1: 1325.28
[900]	valid's l1: 1321.03
[1000]	valid's l1: 1316.86
[1100]	valid's l1: 1314.75
[1200]	valid's l1: 1312.53
Did not meet early stopping. Best iteration is:
[1199]	valid's l1: 1312.52
Lvl_0_Pipe_1_Mod_1_LightGBM fitting and predicting completed


...

[2021-09-26 19:39:35,329] (INFO): Trial 15 finished with value: -1370.4551700190136 and parameters: {'feature_fraction': 0.5914831747534568, 'num_leaves': 194, 'bagging_fraction': 0.8936277429653271, 'min_sum_hessian_in_leaf': 0.166712291358107, 'reg_alpha': 4.9497914616273814e-05, 'reg_lambda': 1.036963650861359e-06}. Best is trial 12 with value: -1361.4943052062647.


Start fitting Lvl_0_Pipe_1_Mod_1_LightGBM ...

===== Start working with fold 0 for Lvl_0_Pipe_1_Mod_1_LightGBM =====

Training until validation scores don't improve for 100 rounds
[100]	valid's l1: 1455.74
[200]	valid's l1: 1410.87
[300]	valid's l1: 1393.92
[400]	valid's l1: 1381.19
[500]	valid's l1: 1376.39
[600]	valid's l1: 1372.29
[700]	valid's l1: 1368.16
[800]	valid's l1: 1364.95
[900]	valid's l1: 1362.54
[1000]	valid's l1: 1359.71
[1100]	valid's l1: 1356.7
[1200]	valid's l1: 1355.69
[1300]	valid's l1: 1354.69
Early stopping, best iteration is:
[1259]	valid's l1: 1354.42

===== Start working with fold 1 for Lvl_0_Pipe_1_Mod_1_LightGBM =====

Training until validation scores don't improve for 100 rounds
[100]	valid's l1: 1339.5
[200]	valid's l1: 1297.19
[300]	valid's l1: 1283.85
[400]	valid's l1: 1276.98
[500]	valid's l1: 1272.61
[600]	valid's l1: 1267.31
[700]	valid's l1: 1264.66
[800]	valid's l1: 1262.48
[900]	valid's l1: 1260.28
[1000]	valid's l1: 1259.25
[1100]	valid's l1: 1258

[2021-09-26 19:45:57,529] (WARNING): C:\Users\User\AppData\Local\Programs\Python\Python37\lib\site-packages\numpy\core\fromnumeric.py:87: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)



Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...

===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LinearL2 =====

Linear model: C = 1e-05 score = -3822.1566178152902
Linear model: C = 5e-05 score = -3817.2438549804688
Linear model: C = 0.0001 score = -3810.884972795759
Linear model: C = 0.0005 score = -3762.898037248884
Linear model: C = 0.001 score = -3707.2527440011163
Linear model: C = 0.005 score = -3392.2609402138846
Linear model: C = 0.01 score = -3172.8490260445733
Linear model: C = 0.05 score = -2691.937402368818
Linear model: C = 0.1 score = -2557.981369875227
Linear model: C = 0.5 score = -2347.4993522513255
Linear model: C = 1 score = -2305.5848647989546
Linear model: C = 5 score = -2263.3669815684725
Linear model: C = 10 score = -2263.3669848038808
Linear model: C = 50 score = -2263.3669331692286
Linear model: C = 100 score = -2263.3668795318604
Linear model: C = 500 score = -2263.3667425842286
Linear model: C = 1000 score = -2263.366698425838
Linear model: C = 5000 sco

KeyboardInterrupt: 

## Step 4. Predict to test data and check scores

In [ ]:
%%time

test_pred = automl.predict(test_data)
logging.info('Prediction for test data:\n{}\nShape = {}'
              .format(test_pred, test_pred.shape))

logging.info('Check scores...')
logging.info('OOF score: {}'.format(mean_absolute_error(train_data[TARGET_NAME].values, oof_pred.data[:, 0])))

CPU times: user 8min 17s, sys: 221 ms, total: 8min 17s
Wall time: 2min 11s


In [ ]:
submission[TARGET_NAME] = test_pred.data[:, 0]
submission.head()

,row_ID,final_price
0,35000,2856.394287
1,35001,5529.273438
2,35002,2622.005859
3,35003,6810.166504
4,35004,4497.337891


## Step 5. Generate submission

In [ ]:
# group train data by car_vin aggregate final price by mode
train_vins = train_data[['car_vin', 'final_price']].groupby('car_vin').agg({'final_price' : 
                                                                            lambda x: x.value_counts()
                                                                            .index[0]}).reset_index()

# copy submission df
sub = submission.copy().set_index('row_ID')
# add car_vin feature from test df
sub = sub.join(test_data[['car_vin', 'row_ID']].set_index('row_ID'), on=sub.index).reset_index()
# join to sub df train_vins df on car_vin feature
sub = sub[['row_ID', 'car_vin']].set_index('car_vin').join(train_vins.set_index('car_vin'), 
                                                            on='car_vin', how='left').reset_index()

# get indices than have not-null car prices
sub_index = sub[sub['final_price'].notnull()].index
# assert not-null car prices of sub df to corresponding prices column of submission df
submission.loc[sub_index, 'final_price'] = sub.loc[sub['final_price'].notnull(), 'final_price']

In [ ]:
submission.to_csv('submission.csv', index = False)